In [1]:
import pandas as pd
import numpy as np
import os
import sys
import itertools
from idd_forecast_mbp import constants as rfc
from idd_forecast_mbp.parquet_functions import read_parquet_with_integer_ids, write_parquet
import glob
cause_map = rfc.cause_map
cause = "dengue"
reference_age_group_id = cause_map[cause]['reference_age_group_id']
reference_sex_id = cause_map[cause]['reference_sex_id']
PROCESSED_DATA_PATH = rfc.PROCESSED_DATA_PATH
as_full_cause_df_path = f'{PROCESSED_DATA_PATH}/as_full_{cause}_df.parquet'
full_2023_hierarchy_path = f"{PROCESSED_DATA_PATH}/full_hierarchy_2023_lsae_1209.parquet"
hierarchy_df = read_parquet_with_integer_ids(full_2023_hierarchy_path)

In [2]:

forecast_years = list(range(2022, 2101))
md_gbd_location_df = hierarchy_df[hierarchy_df['most_detailed_gbd'] == True].copy()
last_year = min(forecast_years)
md_gbd_location_filter = ('location_id', 'in', md_gbd_location_df['location_id'].unique().tolist())
last_year_filter = ('year_id', '==', last_year)
as_md_gbd_dengue_df = read_parquet_with_integer_ids(as_full_cause_df_path,
                                                   filters = [md_gbd_location_filter, last_year_filter])
base_md_gbd_dengue_df = as_md_gbd_dengue_df[(as_md_gbd_dengue_df['age_group_id'] == reference_age_group_id) &
                                              (as_md_gbd_dengue_df['sex_id'] == reference_sex_id)].copy()
base_md_gbd_dengue_df = base_md_gbd_dengue_df.rename(columns={
    'dengue_inc_rate': 'base_dengue_inc_rate'}).reset_index(drop=True)

as_md_gbd_dengue_df = as_md_gbd_dengue_df.merge(base_md_gbd_dengue_df[['location_id', 'year_id', 'base_dengue_inc_rate']],
                                                   on=['location_id', 'year_id'], how='left')
as_md_gbd_dengue_df['rr_inc_as'] = as_md_gbd_dengue_df['dengue_inc_rate'] / as_md_gbd_dengue_df['base_dengue_inc_rate']
# set rr_inc_as to 0 when dengue_inc_rate is 0
as_md_gbd_dengue_df.loc[as_md_gbd_dengue_df['dengue_inc_rate'] == 0, 'rr_inc_as'] = 0

as_md_gbd_dengue_df_path = f"{PROCESSED_DATA_PATH}/as_md_gbd_dengue_df.parquet"
write_parquet(as_md_gbd_dengue_df, as_md_gbd_dengue_df_path)


🗑️ Removed existing file: /mnt/team/idd/pub/forecast-mbp/02-processed_data/as_md_gbd_dengue_df.parquet
✅ Metadata validation passed for /mnt/team/idd/pub/forecast-mbp/02-processed_data/as_md_gbd_dengue_df.parquet


True

In [3]:
as_md_gbd_dengue_df[as_md_gbd_dengue_df['location_id'] == 157]

,location_id,year_id,age_group_id,sex_id,aa_dengue_mort_count,population,aa_population,aa_dengue_mort_rate,dengue_mort_rate,dengue_mort_count,aa_dengue_inc_count,aa_dengue_inc_rate,dengue_inc_rate,dengue_inc_count,base_dengue_inc_rate,rr_inc_as
5850,157,2022,2,1,0.000000,1.019757e+04,3.475722e+07,1.177493e-09,0.000000e+00,0.000000,0.000000,0.005656,0.000000,0.000000,0.002552,0.000000
5851,157,2022,2,2,0.000000,9.609671e+03,3.475722e+07,1.177493e-09,0.000000e+00,0.000000,0.000000,0.005656,0.000000,0.000000,0.002552,0.000000
5852,157,2022,3,1,0.040926,3.026450e+04,3.475722e+07,1.177493e-09,1.014329e-09,0.000031,196599.224481,0.005656,0.002552,77.224022,0.002552,1.000000
5853,157,2022,3,2,0.040926,2.859618e+04,3.475722e+07,1.177493e-09,4.149361e-09,0.000119,196599.224481,0.005656,0.002946,84.257717,0.002552,1.154736
5854,157,2022,6,1,0.040926,2.431392e+06,3.475722e+07,1.177493e-09,2.815800e-10,0.000685,196599.224481,0.005656,0.006326,15381.510288,0.002552,2.479277
5855,157,2022,6,2,0.040926,2.295388e+06,3.475722e+07,1.177493e-09,3.252875e-10,0.000747,196599.224481,0.005656,0.007305,16768.054298,0.002552,2.862910
5856,157,2022,7,1,0.040926,2.299036e+06,3.475722e+07,1.177493e-09,3.118516e-10,0.000717,196599.224481,0.005656,0.006416,14750.961220,0.002552,2.514523
5857,157,2022,7,2,0.040926,2.167343e+06,3.475722e+07,1.177493e-09,1.315007e-10,0.000285,196599.224481,0.005656,0.007409,16057.758422,0.002552,2.903610
5858,157,2022,8,1,0.040926,1.936236e+06,3.475722e+07,1.177493e-09,1.451954e-10,0.000281,196599.224481,0.005656,0.005862,11349.298935,0.002552,2.297163
5859,157,2022,8,2,0.040926,1.854786e+06,3.475722e+07,1.177493e-09,4.004736e-11,0.000074,196599.224481,0.005656,0.006769,12554.151083,0.002552,2.652617


In [16]:
hierarchy_df = read_parquet_with_integer_ids(full_2023_hierarchy_path)
hierarchy_df.loc[hierarchy_df["parent_id"] == 163, 'most_detailed_gbd'] = 1
write_parquet(hierarchy_df, full_2023_hierarchy_path)

✅ Metadata validation passed for /mnt/team/idd/pub/forecast-mbp/02-processed_data/full_hierarchy_2023_lsae_1209.parquet


True